In [1]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
#from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
def load_json(input_file):
    with open(input_file) as f:
    
        reading = json.load(f)
    return reading

train_list = np.array(load_json("trainbiglist.json"))
test_list = np.array(load_json("testbiglist.json"))

train_X = np.expand_dims(train_list,axis=-1)
test_X = np.expand_dims(test_list,axis=-1)

In [ ]:
indices = np.where(arr2d!=0.5)
indexx,indexy = indices[1],indices[0]

#print(indices)

#print(indexx)
#print(indexy)

arr2d[arr2d==0.5] = 0

In [ ]:
print(train_X.shape)
print(test_X.shape)

In [5]:
#build the encoder
inputs = Input((150,150,1))
x = inputs

x = Conv2D(32, (3,3), strides=1, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=-1)(x)

x = Conv2D(16, (3,3), strides=1, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=-1)(x)

volumeSize = K.int_shape(x)
x = Flatten()(x)
latent = Dense(1800)(x)
encoder = Model(inputs, latent, name="encoder")

print(encoder.summary())

ResourceExhaustedError: ignored

In [ ]:
#build the decoder
latentInputs = Input(shape=(1800,))
x = Dense(np.prod(volumeSize[1:]))(latentInputs)
x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

x = Conv2DTranspose(32, (3, 3), strides=1, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=-1)(x)

x = Conv2DTranspose(16, (3, 3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=-1)(x)

x = Conv2DTranspose(32, (3, 3), strides=1, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=-1)(x)

x = Conv2DTranspose(1, (3, 3), padding="same")(x)
outputs = Activation("sigmoid")(x)

decoder = Model(latentInputs, outputs, name="decoder")
print(decoder.summary())

In [ ]:
#build the autoencoder
autoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")
print(autoencoder.summary())

In [ ]:
autoencoder.compile(optimizer=Adam(lr=1e-3), loss="mean_squared_error")

In [ ]:
#train the model
history = autoencoder.fit(
	train_X, train_X,
	validation_data=(test_X, test_X),
	epochs=100)

In [ ]:
autoencoder.save('unsup_autoencoder.h5')

In [ ]:
def draw_grid(out):
  res=out.shape
  plt.imshow(out, cmap = "PiYG_r")
  plt.clim(0.0, 1.0)
  plt.gca().set_xticks(np.arange(-.5, res[1], 2), minor = True)
  plt.gca().set_yticks(np.arange(-.5, res[0], 2), minor = True)
  plt.grid(True, which="minor", color="w", linewidth = .6, alpha = 0.5)
  plt.colorbar()
  plt.show()

In [ ]:
out = train_list[10]
draw_grid(out)

In [ ]:
print(out.shape)
testX=np.expand_dims(out,axis=-1)
print(testX.shape)
testtX = np.reshape(testX, (1,150,150,1))
print(testtX.shape)

In [ ]:
decoded = autoencoder.predict(testtX)

In [ ]:
print(decoded.shape)

In [ ]:
output = np.reshape(decoded, (150,150))

In [ ]:
draw_grid(output)